In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import io
import re
from scipy.interpolate import interp1d

In [ ]:
capnormal=pd.read_csv("HA_capNormalVol.csv")
swaps=pd.read_csv("HA_swaps.csv")
swaption=pd.read_csv("HA_swaptionNormalVol.csv")

In [ ]:
swaps

,Unnamed: 0,Deposit 6M,FRA 1X7,FRA 2X8,FRA 3X9,FRA 4X10,FRA 5X11,FRA 6X12,FRA 7X13,FRA 8X14,...,Swap 9Y,Swap 10Y,Swap 11Y,Swap 12Y,Swap 15Y,Swap 20Y,Swap 25Y,Swap 30Y,Swap 40Y,Swap 50Y
0,2007-01-02,3.857,3.9285,3.9770,4.0155,4.0455,4.0755,4.0975,4.0965,4.1010,...,4.1520,4.1720,4.1870,4.2060,4.2445,4.2840,4.2750,4.2590,4.2160,4.1740
1,2007-01-03,3.860,3.9170,3.9720,4.0145,4.0410,4.0670,4.0820,4.0870,4.0920,...,4.1505,4.1715,4.1885,4.2045,4.2580,4.2845,4.2950,4.2635,4.2205,4.1785
2,2007-01-04,3.865,3.9315,3.9865,4.0225,4.0435,4.0720,4.0915,4.0975,4.1025,...,4.1445,4.1625,4.1815,4.1985,4.2455,4.2755,4.2780,4.2565,4.2145,4.1725
3,2007-01-05,3.864,3.9415,3.9950,4.0410,4.0710,4.0950,4.1220,4.1305,4.1365,...,4.1875,4.2055,4.2235,4.2405,4.2885,4.3175,4.3290,4.2970,4.2550,4.2150
4,2007-01-08,3.874,3.9435,4.0015,4.0435,4.0695,4.0985,4.1225,4.1215,4.1275,...,4.1860,4.2060,4.2260,4.2420,4.2885,4.3200,4.3210,4.3000,4.2590,4.2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3640,2020-04-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.1620,0.2107,NaN,0.1346,0.0610,NaN
3641,2020-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.1935,NaN,NaN,NaN,-0.0085,0.0341,-0.0012,-0.0610,-0.1590,NaN
3642,2019-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5760,0.5300,NaN
3643,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6211,0.5751,NaN


In [ ]:
def schedule(start, end, freq=6):            
    def advance(dt, months):
        try:
            return date(dt.year, dt.month + months, dt.day)
        except:
            return date(dt.year + 1, dt.month - months, dt.day)
    dates = [start]
    while dates[-1] <= end:
        dates.append(advance(dates[-1], 6))
    return dates[:-1]

In [ ]:
#Exercise 1
#creating schedule of dates
start = date(2007,1,15)
maturity = date(2022,1,19)
dates = schedule(start, maturity)

In [ ]:
#Exercise 1
swaps = swaps.rename(columns={'Unnamed: 0': 'Date'}) #changing name of columns
swaps['Date'] = pd.to_datetime(swaps['Date']) #setting the Date columns to a datetime columns
swaps.set_index('Date', inplace=True) #setting the column as the index



In [ ]:
start_1= pd.to_datetime(dates[0])  

In [ ]:
swaps_v1=swaps.loc[start_1].to_frame() #creating a data frame with the rates for january 15th 2007
swaps_v1.rename(columns = {start_1:'par_rate'}, inplace = True)
swaps_v1

,par_rate
Deposit 6M,3.8800
FRA 1X7,3.9520
FRA 2X8,4.0260
FRA 3X9,4.0659
FRA 4X10,4.1048
FRA 5X11,4.1386
FRA 6X12,4.1543
FRA 7X13,4.1606
FRA 8X14,4.1689
FRA 9X15,4.1675


In [ ]:
swaps_v1.reset_index(inplace=True)
swaps_v1

,index,par_rate
0,Deposit 6M,3.8800
1,FRA 1X7,3.9520
2,FRA 2X8,4.0260
3,FRA 3X9,4.0659
4,FRA 4X10,4.1048
5,FRA 5X11,4.1386
6,FRA 6X12,4.1543
7,FRA 7X13,4.1606
8,FRA 8X14,4.1689
9,FRA 9X15,4.1675


In [ ]:
swaps_v1[['instrument', 'maturity']] = swaps_v1['index'].str.split(' ', expand=True) #splitting the index column to have a columns with only the name of the product
swaps_v1

,index,par_rate,instrument,maturity
0,Deposit 6M,3.8800,Deposit,6M
1,FRA 1X7,3.9520,FRA,1X7
2,FRA 2X8,4.0260,FRA,2X8
3,FRA 3X9,4.0659,FRA,3X9
4,FRA 4X10,4.1048,FRA,4X10
5,FRA 5X11,4.1386,FRA,5X11
6,FRA 6X12,4.1543,FRA,6X12
7,FRA 7X13,4.1606,FRA,7X13
8,FRA 8X14,4.1689,FRA,8X14
9,FRA 9X15,4.1675,FRA,9X15


In [ ]:
swaps_v1["maturity"] = swaps_v1["maturity"].apply(lambda x: x + "M" if "X" in x else x) #adding a M to the FRAs maturities
swaps_v1

,index,par_rate,instrument,maturity
0,Deposit 6M,3.8800,Deposit,6M
1,FRA 1X7,3.9520,FRA,1X7M
2,FRA 2X8,4.0260,FRA,2X8M
3,FRA 3X9,4.0659,FRA,3X9M
4,FRA 4X10,4.1048,FRA,4X10M
5,FRA 5X11,4.1386,FRA,5X11M
6,FRA 6X12,4.1543,FRA,6X12M
7,FRA 7X13,4.1606,FRA,7X13M
8,FRA 8X14,4.1689,FRA,8X14M
9,FRA 9X15,4.1675,FRA,9X15M


In [ ]:
swaps_v1['maturity_fra'] = swaps_v1['maturity'].str.split('X').str[1] #getting the maturity of the fra
swaps_v1

,index,par_rate,instrument,maturity,maturity_fra
0,Deposit 6M,3.8800,Deposit,6M,NaN
1,FRA 1X7,3.9520,FRA,1X7M,7M
2,FRA 2X8,4.0260,FRA,2X8M,8M
3,FRA 3X9,4.0659,FRA,3X9M,9M
4,FRA 4X10,4.1048,FRA,4X10M,10M
5,FRA 5X11,4.1386,FRA,5X11M,11M
6,FRA 6X12,4.1543,FRA,6X12M,12M
7,FRA 7X13,4.1606,FRA,7X13M,13M
8,FRA 8X14,4.1689,FRA,8X14M,14M
9,FRA 9X15,4.1675,FRA,9X15M,15M


In [ ]:
swaps_v1.loc[swaps_v1["instrument"] == "FRA", "maturity"] = swaps_v1["maturity_fra"] #getting the new maturity

In [ ]:
swaps_v1

,index,par_rate,instrument,maturity,maturity_fra
0,Deposit 6M,3.8800,Deposit,6M,NaN
1,FRA 1X7,3.9520,FRA,7M,7M
2,FRA 2X8,4.0260,FRA,8M,8M
3,FRA 3X9,4.0659,FRA,9M,9M
4,FRA 4X10,4.1048,FRA,10M,10M
5,FRA 5X11,4.1386,FRA,11M,11M
6,FRA 6X12,4.1543,FRA,12M,12M
7,FRA 7X13,4.1606,FRA,13M,13M
8,FRA 8X14,4.1689,FRA,14M,14M
9,FRA 9X15,4.1675,FRA,15M,15M


In [ ]:
def extract_number(string):
    # Use a regular expression to extract the numeric value from the string
    number = re.findall(r'\d+', string)
    # If the string contains the character 'M', return the number as a float
    # (dividing the number by 12 to convert from months to years)
    if 'M' in string:
        return float(number[0]) / 12
    # Otherwise, return the number as an integer
    else:
        return int(number[0])

In [ ]:
swaps_v1["year_frac"] = swaps_v1["maturity"].apply(extract_number)
swaps_v1

,index,par_rate,instrument,maturity,maturity_fra,year_frac
0,Deposit 6M,3.8800,Deposit,6M,NaN,0.500000
1,FRA 1X7,3.9520,FRA,7M,7M,0.583333
2,FRA 2X8,4.0260,FRA,8M,8M,0.666667
3,FRA 3X9,4.0659,FRA,9M,9M,0.750000
4,FRA 4X10,4.1048,FRA,10M,10M,0.833333
5,FRA 5X11,4.1386,FRA,11M,11M,0.916667
6,FRA 6X12,4.1543,FRA,12M,12M,1.000000
7,FRA 7X13,4.1606,FRA,13M,13M,1.083333
8,FRA 8X14,4.1689,FRA,14M,14M,1.166667
9,FRA 9X15,4.1675,FRA,15M,15M,1.250000


In [ ]:
swaps_v1.set_index('index',inplace=True)

In [ ]:
swaps_v1

,par_rate,instrument,maturity,maturity_fra,year_frac
index,,,,,
Deposit 6M,3.8800,Deposit,6M,NaN,0.500000
FRA 1X7,3.9520,FRA,7M,7M,0.583333
FRA 2X8,4.0260,FRA,8M,8M,0.666667
FRA 3X9,4.0659,FRA,9M,9M,0.750000
FRA 4X10,4.1048,FRA,10M,10M,0.833333
FRA 5X11,4.1386,FRA,11M,11M,0.916667
FRA 6X12,4.1543,FRA,12M,12M,1.000000
FRA 7X13,4.1606,FRA,13M,13M,1.083333
FRA 8X14,4.1689,FRA,14M,14M,1.166667


In [ ]:
forward=swaps_v1.loc['FRA 6X12', 'par_rate']

In [ ]:
one_year_rate=(((1+swaps_v1.loc['Deposit 6M', 'par_rate'])**swaps_v1.loc['Deposit 6M', 'year_frac'])*(1+forward)**(1-swaps_v1.loc['Deposit 6M', 'year_frac']))-1

In [ ]:
one_year_rate

4.015275067232105

In [ ]:
yield_curve = swaps_v1[swaps_v1.index.str.contains("FRA") == False]

In [ ]:
one_year_rate_frame = pd.DataFrame([[one_year_rate,'-','1Y','-',1.0]],
     index=['Swap 1Y'],
     columns=['par_rate', 'instrument','maturity','maturity_fra','year_frac'])
one_year_rate_frame

,par_rate,instrument,maturity,maturity_fra,year_frac
Swap 1Y,4.015275,-,1Y,-,1.0


In [ ]:
yield_curve=yield_curve.append(one_year_rate_frame)

In [ ]:
yield_curve.set_index('year_frac',inplace=True)

In [ ]:
yield_curve


,par_rate,instrument,maturity,maturity_fra
year_frac,,,,
0.5,3.880000,Deposit,6M,NaN
2.0,4.169000,Swap,2Y,NaN
3.0,4.172500,Swap,3Y,NaN
4.0,4.175500,Swap,4Y,NaN
5.0,4.184500,Swap,5Y,NaN
6.0,4.202500,Swap,6Y,NaN
7.0,4.218500,Swap,7Y,NaN
8.0,4.242500,Swap,8Y,NaN
9.0,4.264500,Swap,9Y,NaN


In [ ]:
yield_curve.loc[0.5, 'df'] = 1 / (1 + yield_curve.loc[0.5, 'par_rate']/100 * 0.5)
yield_curve.loc[0.5, 'zero_rate'] = yield_curve.loc[0.5, 'par_rate']/100
yield_curve

,par_rate,instrument,maturity,maturity_fra,df,zero_rate
year_frac,,,,,,
0.5,3.880000,Deposit,6M,NaN,0.980969,0.0388
2.0,4.169000,Swap,2Y,NaN,NaN,NaN
3.0,4.172500,Swap,3Y,NaN,NaN,NaN
4.0,4.175500,Swap,4Y,NaN,NaN,NaN
5.0,4.184500,Swap,5Y,NaN,NaN,NaN
6.0,4.202500,Swap,6Y,NaN,NaN,NaN
7.0,4.218500,Swap,7Y,NaN,NaN,NaN
8.0,4.242500,Swap,8Y,NaN,NaN,NaN
9.0,4.264500,Swap,9Y,NaN,NaN,NaN


In [ ]:
yield_curve.loc[1, 'df'] = 1 / (1 + yield_curve.loc[1, 'par_rate']/100 * 1)
yield_curve.loc[1, 'zero_rate'] = yield_curve.loc[1, 'par_rate']/100


In [ ]:
yield_curve=yield_curve.sort_index()
yield_curve

,par_rate,instrument,maturity,maturity_fra,df,zero_rate
year_frac,,,,,,
0.5,3.880000,Deposit,6M,NaN,0.980969,0.038800
1.0,4.015275,-,1Y,-,0.961397,0.040153
2.0,4.169000,Swap,2Y,NaN,NaN,NaN
3.0,4.172500,Swap,3Y,NaN,NaN,NaN
4.0,4.175500,Swap,4Y,NaN,NaN,NaN
5.0,4.184500,Swap,5Y,NaN,NaN,NaN
6.0,4.202500,Swap,6Y,NaN,NaN,NaN
7.0,4.218500,Swap,7Y,NaN,NaN,NaN
8.0,4.242500,Swap,8Y,NaN,NaN,NaN


In [ ]:
yield_curve.loc[2, 'df'] = ((1 - yield_curve.loc[2, 'par_rate']/100) * yield_curve.loc[1, 'df']) / ((yield_curve.loc[2, 'par_rate']/100) + 1)
yield_curve.loc[2, 'zero_rate'] = np.power(1 / yield_curve.loc[2, 'df'], 1 / 2 ) - 1
yield_curve

,par_rate,instrument,maturity,maturity_fra,df,zero_rate
year_frac,,,,,,
0.5,3.880000,Deposit,6M,NaN,0.980969,0.038800
1.0,4.015275,-,1Y,-,0.961397,0.040153
2.0,4.169000,Swap,2Y,NaN,0.884444,0.063322
3.0,4.172500,Swap,3Y,NaN,0.921439,0.027648
4.0,4.175500,Swap,4Y,NaN,0.882907,0.031624
5.0,4.184500,Swap,5Y,NaN,0.845656,0.034097
6.0,4.202500,Swap,6Y,NaN,0.809411,0.035870
7.0,4.218500,Swap,7Y,NaN,0.774485,0.037183
8.0,4.242500,Swap,8Y,NaN,0.740312,0.038301


In [ ]:
for year in range(3,19):
    dfs = yield_curve.iloc[1:year-1].df.sum()
    rate = yield_curve.iloc[year, 0]/100
    df = (1 - rate * dfs) / (1+rate)
    yield_curve.iloc[year, 4] = df
    yield_curve.iloc[year, 5] = np.power(1 / df, 1 / year ) - 1
yield_curve

,par_rate,instrument,maturity,maturity_fra,df,zero_rate
year_frac,,,,,,
0.5,3.880000,Deposit,6M,NaN,0.980969,0.038800
1.0,4.015275,-,1Y,-,0.961397,0.040153
2.0,4.169000,Swap,2Y,NaN,0.884444,0.063322
3.0,4.172500,Swap,3Y,NaN,0.921439,0.027648
4.0,4.175500,Swap,4Y,NaN,0.885935,0.030741
5.0,4.184500,Swap,5Y,NaN,0.848690,0.033357
6.0,4.202500,Swap,6Y,NaN,0.812335,0.035247
7.0,4.218500,Swap,7Y,NaN,0.777297,0.036646
8.0,4.242500,Swap,8Y,NaN,0.743021,0.037827


In [ ]:
# Start date
start_date = date(2007, 1, 19)

# Add six months and keep the same day
loan_dates=[]
for i in range(30):
  resulting_date = start_date + relativedelta(months=+6)
  if resulting_date.weekday()==5:
    resulting_date=resulting_date+timedelta(days=2)
  if resulting_date.weekday()==6:
    resulting_date=resulting_date+timedelta(days=1)
  loan_dates.append(resulting_date)
  start_date=date(resulting_date.year,resulting_date.month,19)
loan_dates

[datetime.date(2007, 7, 19),
 datetime.date(2008, 1, 21),
 datetime.date(2008, 7, 21),
 datetime.date(2009, 1, 19),
 datetime.date(2009, 7, 20),
 datetime.date(2010, 1, 19),
 datetime.date(2010, 7, 19),
 datetime.date(2011, 1, 19),
 datetime.date(2011, 7, 19),
 datetime.date(2012, 1, 19),
 datetime.date(2012, 7, 19),
 datetime.date(2013, 1, 21),
 datetime.date(2013, 7, 19),
 datetime.date(2014, 1, 20),
 datetime.date(2014, 7, 21),
 datetime.date(2015, 1, 19),
 datetime.date(2015, 7, 20),
 datetime.date(2016, 1, 19),
 datetime.date(2016, 7, 19),
 datetime.date(2017, 1, 19),
 datetime.date(2017, 7, 19),
 datetime.date(2018, 1, 19),
 datetime.date(2018, 7, 19),
 datetime.date(2019, 1, 21),
 datetime.date(2019, 7, 19),
 datetime.date(2020, 1, 20),
 datetime.date(2020, 7, 20),
 datetime.date(2021, 1, 19),
 datetime.date(2021, 7, 19),
 datetime.date(2022, 1, 19)]

In [ ]:
start = date(2007,1,15)

In [ ]:
nr_days=[]
for a in range(0,len(loan_dates)):
  nr_days.append((loan_dates[a]-start).days/365)
nr_days

[0.5068493150684932,
 1.0164383561643835,
 1.515068493150685,
 2.0136986301369864,
 2.5123287671232877,
 3.0136986301369864,
 3.5095890410958903,
 4.013698630136986,
 4.50958904109589,
 5.013698630136986,
 5.512328767123288,
 6.021917808219178,
 6.512328767123288,
 7.019178082191781,
 7.517808219178082,
 8.016438356164384,
 8.515068493150684,
 9.016438356164384,
 9.515068493150684,
 10.01917808219178,
 10.515068493150684,
 11.01917808219178,
 11.515068493150684,
 12.024657534246575,
 12.515068493150684,
 13.021917808219179,
 13.520547945205479,
 14.021917808219179,
 14.517808219178082,
 15.021917808219179]

In [ ]:
y=np.array(yield_curve.zero_rate.values)
x=np.array(yield_curve.index.values)
y_interp = interp1d(x,y)
 
# Finding the interpolation
interpolated_rates=[]
for a in nr_days:
  interpolate_x=a
  interpolated_rates.append(float(y_interp(interpolate_x)))
interpolated_rates

[0.038818530831127684,
 0.040533615003719436,
 0.05208649972280382,
 0.06283331715227407,
 0.045045352529404184,
 0.02769069820299443,
 0.02922434809204035,
 0.030776880729988677,
 0.0320738961958739,
 0.03338248057753671,
 0.03432526163071664,
 0.035277976200834005,
 0.03596385899538394,
 0.03666855737154492,
 0.03725738792727688,
 0.03784273609832302,
 0.03832593431858485,
 0.0388097439211699,
 0.03923095436852557,
 0.03965452062784584,
 0.040014646697386434,
 0.040378904126771725,
 0.04069151561392228,
 0.04100479209586598,
 0.041155421310784895,
 0.04131109954966758,
 0.04146425327656838,
 0.041618248507463144,
 0.041770560730369985,
 0.04192213132799076]

In [ ]:
len(interpolated_rates)

30

In [ ]:
notional=1748653.15
installemnts=np.full(30,1748653.15)
int_pay=installemnts*(interpolated_rates)
int_pay[len(int_pay)-1]=notional*(1+interpolated_rates[len(interpolated_rates)-1])
int_pay

array([  67880.14621622,   70879.23355714,   91081.22181276,
        109873.67796327,   78768.6975934 ,   48421.42663837,
         51103.24834784,   53818.08943567,   56086.11961569,
         58374.37981672,   60022.97687513,   61688.94420921,
         62888.31531843,   64120.58835371,   65150.2487598 ,
         66173.81968295,   67018.76577289,   67864.78095845,
         68601.33193403,   69342.00240762,   69971.73799352,
         70608.69789483,   71155.34695656,   71703.15886353,
         71966.55711468,   72238.78435749,   72506.59710447,
         72775.88135006,   73042.22259843, 1821960.4170014 ])